In [5]:
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.feature import hog,local_binary_pattern
import glob
import numpy as np
from sklearn import svm
from skimage import feature
from sklearn.metrics import classification_report
from skimage.color import rgb2gray

# Read Data

In [6]:
# import from drive
# run once
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [7]:
# import training data
live_train = glob.glob('data/TestingTtrain_live/*.png') #positive
spoof_train = glob.glob('data/train_spoof/*.png') # negative

# import testing data
live_test = glob.glob('data/test_live/*.png') # positive
spoof_test = glob.glob('data/test_spoof/*.png') # negative

In [8]:
# initialize empty lists to append data
live_train_images = []
spoof_train_images = []
live_test_images = []
spoof_test_images = []

# read images from given path and append to the variavles
for path in live_train: live_train_images.append(imread(path))
for path in spoof_train: spoof_train_images.append(imread(path))
for path in live_test: live_test_images.append(imread(path))
for path in spoof_test: spoof_test_images.append(imread(path))

In [12]:
# convert images to np arrays
live_train_images, spoof_train_images = np.asarray(live_train_images), np.asarray(spoof_train_images)
live_test_images, spoof_test_images = np.asarray(live_test_images), np.asarray(spoof_test_images)

In [13]:
# change the type of feature you want to use here
feature_type = "lbp" #hog or lbp

# initialize empty lists to append features
live_train_features, spoof_train_features = [], []
live_test_features, spoof_test_features = [], []

if(feature_type == "hog"):

    # calculate and store hog features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_train_features.append(hog(live_img, feature_vector = True))
        spoof_train_features.append(hog(spoof_img, feature_vector = True))

    # calculate and store hog features of testing data
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        live_test_features.append(hog(live_img, feature_vector = True))
        spoof_test_features.append(hog(spoof_img, feature_vector = True))

if(feature_type == "lbp"):

# parameters copied from scikit docs of lbp
    METHOD = 'uniform'
    R = 3
    P = 8 * R

    # calculate and store lbp features of training data
    for live_img, spoof_img in zip(live_train_images, spoof_train_images):
        live_img = rgb2gray(live_img)
        spoof_img = rgb2gray(spoof_img)
        live_train_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_train_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

    # calculate and store lbp features of training data
    for live_img, spoof_img in zip(live_test_images, spoof_test_images):
        # convert rgb to gray
        live_img = rgb2gray(live_img)
        # convert rgb to gray
        spoof_img = rgb2gray(spoof_img)
        live_test_features.append(local_binary_pattern(live_img, P, R, METHOD).flatten())
        spoof_test_features.append(local_binary_pattern(spoof_img, P, R, METHOD).flatten())

In [14]:
# define labels 1 for live and 0 for spoof
# i.e. create a list of 200 1's and 200 0's
labels_train = [[1]]*len(live_train_features) + [[0]]*len(spoof_train_features)

# merge both positive and negative training data
full_train = live_train_features + spoof_train_features

# assign labels for all test data, 1 for positive, 0 for negative
labels_test = [[1]]*len(live_test_features) + [[0]]*len(spoof_test_features)

# merge both positive and negative training data
full_test = live_test_features + spoof_test_features